# Imports and installment

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna 
from ydata_profiling import ProfileReport
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error , mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
import joblib
import warnings
from xgboost import XGBRegressor
from gplearn.genetic import SymbolicRegressor
from catboost import CatBoostRegressor
#from lightgbm import LGBMRegressor
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
import json 


# Data exploration and Dataset Creation


### Data sources and target


The dataset consists of two primary sources of information: **electrical consumption, generation, and pricing data**, as well as **weather data**. This data spans four years and covers various energy sources along with meteorological conditions. Below is a detailed description of both data sources and the meaning of each feature (column).

#### 1. **Energy Data**
The energy-related data was retrieved from the **ENTSOE** (European Network of Transmission System Operators for Electricity) public portal, which provides data from Transmission Service Operators (TSOs) across Europe, and **Red Eléctrica de España (REE)**, the Spanish TSO, which provides pricing and settlement data.

The energy dataset provides a comprehensive view of electricity generation, consumption, and pricing across different energy sources in Spain. Each column is described as follows:

- **Time**: A `Datetime` index localized to **CET (Central European Time)**, which serves as the timestamp for each record.
- **Generation biomass**: Electricity generated from **biomass** in megawatts (MW).
- **Generation fossil brown coal/lignite**: Electricity generated from **brown coal or lignite** in MW.
- **Generation fossil coal-derived gas**: Electricity generated from **coal-derived gas** in MW.
- **Generation fossil gas**: Electricity generated from **natural gas** in MW.
- **Generation fossil hard coal**: Electricity generated from **hard coal** in MW.
- **Generation fossil oil**: Electricity generated from **oil** in MW.
- **Generation fossil oil shale**: Electricity generated from **oil shale** in MW.
- **Generation fossil peat**: Electricity generated from **peat** in MW.
- **Generation geothermal**: Electricity generated from **geothermal sources** in MW.
- **Generation hydro pumped storage aggregated**: Total electricity generated from **pumped hydroelectric storage** in MW.
- **Generation hydro pumped storage consumption**: **Consumption of electricity** used to pump water into hydroelectric storage in MW.
- **Generation hydro run-of-river and poundage**: Electricity generated from **run-of-river hydroelectric plants** in MW.
- **Generation hydro water reservoir**: Electricity generated from **reservoir-based hydroelectric plants** in MW.
- **Generation marine**: Electricity generated from **marine (ocean) energy** sources in MW.
- **Generation nuclear**: Electricity generated from **nuclear power plants** in MW.
- **Generation other**: Miscellaneous generation from other unspecified sources in MW.
- **Generation other renewable**: Electricity generated from **other renewable energy sources** in MW.
- **Generation solar**: Electricity generated from **solar power** in MW.
- **Generation waste**: Electricity generated from **waste materials** in MW.
- **Generation wind offshore**: Electricity generated from **offshore wind farms** in MW.
- **Generation wind onshore**: Electricity generated from **onshore wind farms** in MW.
- **Forecast solar day ahead**: **Forecasted solar generation** for the next day in MW.
- **Forecast wind offshore day ahead**: **Forecasted offshore wind generation** for the next day in MW.
- **Forecast wind onshore day ahead**: **Forecasted onshore wind generation** for the next day in MW.
- **Total load forecast**: **Forecasted total electricity demand** in MW.
- **Total load actual**: **Actual total electricity demand** in MW.
- **Price day ahead**: **Forecasted price** of electricity in **EUR/MWh** (Euros per megawatt-hour).
- **Price actual**: **Actual market price** of electricity in **EUR/MWh**.

#### 2. **Weather Data**
The dataset also includes weather data that is integrated with the energy dataset. The weather information for various cities includes temperature, humidity, wind, rain, and other meteorological conditions that can impact electricity consumption and generation (especially renewable sources like wind and solar).

The weather-related columns are as follows:

- **city_name**: The name of the city where the weather data was recorded.
- **temp**: The current **temperature** in degrees Celsius.
- **temp_min**: The **minimum temperature** recorded in degrees Celsius.
- **temp_max**: The **maximum temperature** recorded in degrees Celsius.
- **pressure**: The **atmospheric pressure** in hPa (hectopascals).
- **humidity**: The **humidity level** as a percentage.
- **wind_speed**: The **wind speed** in meters per second (m/s).
- **wind_deg**: The **wind direction** in degrees (meteorological degrees from 0-360°).
- **rain_1h**: The amount of **rainfall** in millimeters (mm) during the last 1 hour.
- **rain_3h**: The amount of **rainfall** in millimeters (mm) during the last 3 hours.
- **snow_3h**: The amount of **snowfall** in millimeters (mm) during the last 3 hours.
- **clouds_all**: The percentage of the sky covered by **clouds**.
- **weather_id**: The **weather condition ID** (numeric code for different weather types, based on OpenWeatherMap API).
- **weather_main**: A brief **description of the weather** (e.g., clear, cloudy, rainy).
- **weather_description**: A more detailed **description of the weather** (e.g., light rain, overcast clouds).
- **weather_icon**: The icon code used to visually represent the **current weather condition**.


#### 3. **Europe  Brent Spot Price FOB**    
The Brent Spot Price FOB (Free on Board) refers to the daily price of Brent crude oil, which is a major benchmark for oil prices globally. Brent crude is a type of light, sweet crude oil extracted from the North Sea and is one of the most widely used standards for oil pricing, especially in Europe and other parts of the world. We included also this feature as comparable energy sources as is done in the paper.

#### Task 
Our goal is to predict electricity prices 6 hours ahead, instead of  24-hour forecast. This shorter prediction horizon is chosen to capture rapid market changes while maintaining a robust dataset. The features used are very similar to those in the paper, allowing for a feature-based learning approach (rather than time series learning) as outlined in the paper.

### Data Exploration and Data cleaning

#### Energy Data analysis

In [2]:
energy_data = pd.read_csv('./Data/energy_dataset.csv')
energy_data['time'] = pd.to_datetime(energy_data['time'], utc=True)

In [3]:
energy_data.describe()

,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
count,35045.000000,35046.000000,35046.0,35046.000000,35046.000000,35045.000000,35046.0,35046.0,35046.0,0.0,...,35045.000000,35046.0,35046.000000,35064.000000,0.0,35064.000000,35064.000000,35028.000000,35064.000000,35064.000000
mean,383.513540,448.059208,0.0,5622.737488,4256.065742,298.319789,0.0,0.0,0.0,NaN,...,269.452133,0.0,5464.479769,1439.066735,NaN,5471.216689,28712.129962,28696.939905,49.874341,57.884023
std,85.353943,354.568590,0.0,2201.830478,1961.601013,52.520673,0.0,0.0,0.0,NaN,...,50.195536,0.0,3213.691587,1677.703355,NaN,3176.312853,4594.100854,4574.987950,14.618900,14.204083
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,NaN,...,0.000000,0.0,0.000000,0.000000,NaN,237.000000,18105.000000,18041.000000,2.060000,9.330000
25%,333.000000,0.000000,0.0,4126.000000,2527.000000,263.000000,0.0,0.0,0.0,NaN,...,240.000000,0.0,2933.000000,69.000000,NaN,2979.000000,24793.750000,24807.750000,41.490000,49.347500
50%,367.000000,509.000000,0.0,4969.000000,4474.000000,300.000000,0.0,0.0,0.0,NaN,...,279.000000,0.0,4849.000000,576.000000,NaN,4855.000000,28906.000000,28901.000000,50.520000,58.020000
75%,433.000000,757.000000,0.0,6429.000000,5838.750000,330.000000,0.0,0.0,0.0,NaN,...,310.000000,0.0,7398.000000,2636.000000,NaN,7353.000000,32263.250000,32192.000000,60.530000,68.010000
max,592.000000,999.000000,0.0,20034.000000,8359.000000,449.000000,0.0,0.0,0.0,NaN,...,357.000000,0.0,17436.000000,5836.000000,NaN,17430.000000,41390.000000,41015.000000,101.990000,116.800000


In [4]:
missing_energy = energy_data.isnull().sum() / len(energy_data) * 100
missing_energy

time                                             0.000000
generation biomass                               0.054187
generation fossil brown coal/lignite             0.051335
generation fossil coal-derived gas               0.051335
generation fossil gas                            0.051335
generation fossil hard coal                      0.051335
generation fossil oil                            0.054187
generation fossil oil shale                      0.051335
generation fossil peat                           0.051335
generation geothermal                            0.051335
generation hydro pumped storage aggregated     100.000000
generation hydro pumped storage consumption      0.054187
generation hydro run-of-river and poundage       0.054187
generation hydro water reservoir                 0.051335
generation marine                                0.054187
generation nuclear                               0.048483
generation other                                 0.051335
generation oth

In [5]:
energy_data = energy_data.loc[:, energy_data.isnull().mean() < 0.5]
energy_data['date'] = energy_data['time'].dt.date
energy_data['time'] = energy_data['time'].dt.time
cols = energy_data.columns.tolist()
cols = [col for col in cols if col not in ['date', 'time']]
start_cols = ['date', 'time']
cols = start_cols + cols
energy_data = energy_data[cols]




In [6]:

energy_data['time'] = energy_data['time'].apply(lambda x: x.strftime('%H:%M:%S'))

energy_data

,date,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,...,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2014-12-31,23:00:00,447.0,329.0,0.0,4844.0,4821.0,162.0,0.0,0.0,...,49.0,196.0,0.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01,00:00:00,449.0,328.0,0.0,5196.0,4755.0,158.0,0.0,0.0,...,50.0,195.0,0.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01,01:00:00,448.0,323.0,0.0,4857.0,4581.0,157.0,0.0,0.0,...,50.0,196.0,0.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01,02:00:00,438.0,254.0,0.0,4314.0,4131.0,160.0,0.0,0.0,...,50.0,191.0,0.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01,03:00:00,428.0,187.0,0.0,4130.0,3840.0,156.0,0.0,0.0,...,42.0,189.0,0.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31,18:00:00,297.0,0.0,0.0,7634.0,2628.0,178.0,0.0,0.0,...,85.0,277.0,0.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-31,19:00:00,296.0,0.0,0.0,7241.0,2566.0,174.0,0.0,0.0,...,33.0,280.0,0.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-31,20:00:00,292.0,0.0,0.0,7025.0,2422.0,168.0,0.0,0.0,...,31.0,286.0,0.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-31,21:00:00,293.0,0.0,0.0,6562.0,2293.0,163.0,0.0,0.0,...,31.0,287.0,0.0,3586.0,29.0,3273.0,25450.0,25801.0,63.93,69.89


In [7]:
# check for duplicates in the data and time columns couple
energy_data.duplicated(subset=['date', 'time']).sum()


0

In [8]:
# chek for the column with zero or one unique value
unique_energy = energy_data.nunique()
unique_energy

date                                            1462
time                                              24
generation biomass                               423
generation fossil brown coal/lignite             956
generation fossil coal-derived gas                 1
generation fossil gas                           8297
generation fossil hard coal                     7266
generation fossil oil                            321
generation fossil oil shale                        1
generation fossil peat                             1
generation geothermal                              1
generation hydro pumped storage consumption     3311
generation hydro run-of-river and poundage      1684
generation hydro water reservoir                7029
generation marine                                  1
generation nuclear                              2388
generation other                                 103
generation other renewable                        78
generation solar                              

In [9]:
# drop all columns with one unique value
energy_data = energy_data.loc[:, energy_data.nunique() > 1]
energy_data

,date,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,...,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2014-12-31,23:00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,...,73.0,49.0,196.0,6378.0,17.0,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01,00:00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,...,71.0,50.0,195.0,5890.0,16.0,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01,01:00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,...,73.0,50.0,196.0,5461.0,8.0,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01,02:00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,...,75.0,50.0,191.0,5238.0,2.0,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01,03:00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,...,74.0,42.0,189.0,4935.0,9.0,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31,18:00:00,297.0,0.0,7634.0,2628.0,178.0,1.0,1135.0,4836.0,...,95.0,85.0,277.0,3113.0,96.0,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-31,19:00:00,296.0,0.0,7241.0,2566.0,174.0,1.0,1172.0,3931.0,...,95.0,33.0,280.0,3288.0,51.0,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-31,20:00:00,292.0,0.0,7025.0,2422.0,168.0,50.0,1148.0,2831.0,...,94.0,31.0,286.0,3503.0,36.0,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-31,21:00:00,293.0,0.0,6562.0,2293.0,163.0,108.0,1128.0,2068.0,...,93.0,31.0,287.0,3586.0,29.0,3273.0,25450.0,25801.0,63.93,69.89


In [10]:
# drop columns price day ahead and otal load forecast
energy_data = energy_data.drop(['price day ahead', 'total load forecast'], axis=1)
energy_data


,date,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load actual,price actual
0,2014-12-31,23:00:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,43.0,73.0,49.0,196.0,6378.0,17.0,6436.0,25385.0,65.41
1,2015-01-01,00:00:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,43.0,71.0,50.0,195.0,5890.0,16.0,5856.0,24382.0,64.92
2,2015-01-01,01:00:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,43.0,73.0,50.0,196.0,5461.0,8.0,5454.0,22734.0,64.48
3,2015-01-01,02:00:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,43.0,75.0,50.0,191.0,5238.0,2.0,5151.0,21286.0,59.32
4,2015-01-01,03:00:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,43.0,74.0,42.0,189.0,4935.0,9.0,4861.0,20264.0,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31,18:00:00,297.0,0.0,7634.0,2628.0,178.0,1.0,1135.0,4836.0,6073.0,63.0,95.0,85.0,277.0,3113.0,96.0,3253.0,30653.0,77.02
35060,2018-12-31,19:00:00,296.0,0.0,7241.0,2566.0,174.0,1.0,1172.0,3931.0,6074.0,62.0,95.0,33.0,280.0,3288.0,51.0,3353.0,29735.0,76.16
35061,2018-12-31,20:00:00,292.0,0.0,7025.0,2422.0,168.0,50.0,1148.0,2831.0,6076.0,61.0,94.0,31.0,286.0,3503.0,36.0,3404.0,28071.0,74.30
35062,2018-12-31,21:00:00,293.0,0.0,6562.0,2293.0,163.0,108.0,1128.0,2068.0,6075.0,61.0,93.0,31.0,287.0,3586.0,29.0,3273.0,25801.0,69.89


In [11]:
# hour to int
energy_data['time'] =pd.to_datetime(energy_data['time'], format='%H:%M:%S').dt.hour
energy_data['time'] = energy_data['time'].astype(int)
# enegy data date as datetime




In [12]:
energy_data['date'] = pd.to_datetime(energy_data['date'])

In [13]:
energy_data.dtypes

date                                           datetime64[ns]
time                                                    int64
generation biomass                                    float64
generation fossil brown coal/lignite                  float64
generation fossil gas                                 float64
generation fossil hard coal                           float64
generation fossil oil                                 float64
generation hydro pumped storage consumption           float64
generation hydro run-of-river and poundage            float64
generation hydro water reservoir                      float64
generation nuclear                                    float64
generation other                                      float64
generation other renewable                            float64
generation solar                                      float64
generation waste                                      float64
generation wind onshore                               float64
forecast

In [96]:
# create report for the energy data with ydata profiling    

#profile = ProfileReport(energy_data, title=' Profiling Report', explorative=True)
#profile.to_file("energy_data_report.html")
#profile

#### Whater Data analysis

In [15]:
weather_data = pd.read_csv('./Data/weather_features.csv')
weather_data['dt_iso'] = pd.to_datetime(weather_data['dt_iso'], utc=True)
weather_data

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2014-12-31 23:00:00+00:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
1,2015-01-01 00:00:00+00:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2,2015-01-01 01:00:00+00:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
3,2015-01-01 02:00:00+00:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
4,2015-01-01 03:00:00+00:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178391,2018-12-31 18:00:00+00:00,Seville,287.760,287.150,288.150,1028,54,3,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178392,2018-12-31 19:00:00+00:00,Seville,285.760,285.150,286.150,1029,62,3,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178393,2018-12-31 20:00:00+00:00,Seville,285.150,285.150,285.150,1028,58,4,50,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178394,2018-12-31 21:00:00+00:00,Seville,284.150,284.150,284.150,1029,57,4,60,0.0,0.0,0.0,0,800,clear,sky is clear,01n


In [16]:
weather_data['date'] = weather_data['dt_iso'].dt.date
weather_data['time'] = weather_data['dt_iso'].dt.time

weather_data['time'] = weather_data['time'].apply(lambda x: x.strftime('%H:%M:%S'))
weather_data['time'] = pd.to_datetime(weather_data['time'], format='%H:%M:%S').dt.hour
weather_data['time'] = weather_data['time'].astype(int)
cols = weather_data.columns.tolist()
cols = [col for col in cols if col not in ['date', 'time']]
start_cols = ['date', 'time']
cols = start_cols + cols
weather_data = weather_data[cols]
weather_data = weather_data.drop(['dt_iso'], axis=1)
weather_data['date'] = pd.to_datetime(weather_data['date'])
weather_data['date'] = weather_data['date'].dt.date
weather_data['date'] = pd.to_datetime(weather_data['date'])


In [17]:
# drop all the data in the weather data that the city name is not Madrid
weather_data = weather_data[weather_data['city_name'] == 'Madrid']
# drop the  column city_name
weather_data = weather_data.drop(columns=['city_name'])


In [18]:
# check values with opne unique value
unique_weather = weather_data.nunique()
unique_weather


date                   1462
time                     24
temp                   7272
temp_min               4370
temp_max               4420
pressure                115
humidity                 96
wind_speed               18
wind_deg                361
rain_1h                   4
rain_3h                  38
snow_3h                   2
clouds_all               88
weather_id               30
weather_main              9
weather_description      32
weather_icon             23
dtype: int64

In [19]:
# drop duplicates in the weather data  date and time    
weather_data.duplicated(subset=['date', 'time']).sum()
    

1203

In [20]:
# drop duplicates in the weather data  date and time
weather_data = weather_data.drop_duplicates(subset=['date', 'time'])
weather_data

,date,time,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
35145,2014-12-31,23,267.325,267.325,267.325,971,63,1,309,0.0,0.0,0.0,0,800,clear,sky is clear,01n
35146,2015-01-01,0,267.325,267.325,267.325,971,63,1,309,0.0,0.0,0.0,0,800,clear,sky is clear,01n
35147,2015-01-01,1,266.186,266.186,266.186,971,64,1,273,0.0,0.0,0.0,0,800,clear,sky is clear,01n
35148,2015-01-01,2,266.186,266.186,266.186,971,64,1,273,0.0,0.0,0.0,0,800,clear,sky is clear,01n
35149,2015-01-01,3,266.186,266.186,266.186,971,64,1,273,0.0,0.0,0.0,0,800,clear,sky is clear,01n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71407,2018-12-31,18,283.560,282.150,285.150,1030,88,1,280,0.0,0.0,0.0,0,800,clear,sky is clear,01n
71408,2018-12-31,19,280.120,278.150,281.150,1031,52,1,260,0.0,0.0,0.0,0,800,clear,sky is clear,01n
71409,2018-12-31,20,278.150,278.150,278.150,1030,65,1,340,0.0,0.0,0.0,0,800,clear,sky is clear,01n
71410,2018-12-31,21,276.570,276.150,277.150,1031,69,2,340,0.0,0.0,0.0,0,800,clear,sky is clear,01n


In [21]:
# proile the weather data
#profile = ProfileReport(weather_data, title=' Profiling Report', explorative=True)
#profile.to_file("weather_data_report.html")
#profile

In [110]:
# apply the one hot encoding to the weather data weather_main      objectweather_description         weather_icon

weather_data = pd.get_dummies(weather_data, columns=['weather_main', 'weather_description', 'weather_icon'])
weather_data

,date,time,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,...,weather_icon_09n,weather_icon_10,weather_icon_10d,weather_icon_10n,weather_icon_11d,weather_icon_11n,weather_icon_13d,weather_icon_13n,weather_icon_50d,weather_icon_50n
35145,2014-12-31,23,267.325,267.325,267.325,971,63,1,309,0.0,...,False,False,False,False,False,False,False,False,False,False
35146,2015-01-01,0,267.325,267.325,267.325,971,63,1,309,0.0,...,False,False,False,False,False,False,False,False,False,False
35147,2015-01-01,1,266.186,266.186,266.186,971,64,1,273,0.0,...,False,False,False,False,False,False,False,False,False,False
35148,2015-01-01,2,266.186,266.186,266.186,971,64,1,273,0.0,...,False,False,False,False,False,False,False,False,False,False
35149,2015-01-01,3,266.186,266.186,266.186,971,64,1,273,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71407,2018-12-31,18,283.560,282.150,285.150,1030,88,1,280,0.0,...,False,False,False,False,False,False,False,False,False,False
71408,2018-12-31,19,280.120,278.150,281.150,1031,52,1,260,0.0,...,False,False,False,False,False,False,False,False,False,False
71409,2018-12-31,20,278.150,278.150,278.150,1030,65,1,340,0.0,...,False,False,False,False,False,False,False,False,False,False
71410,2018-12-31,21,276.570,276.150,277.150,1031,69,2,340,0.0,...,False,False,False,False,False,False,False,False,False,False


In [111]:
# check for missing values in the weather data
missing_weather = weather_data.isnull().sum() / len(weather_data) * 100
missing_weather


date                0.0
time                0.0
temp                0.0
temp_min            0.0
temp_max            0.0
                   ... 
weather_icon_11n    0.0
weather_icon_13d    0.0
weather_icon_13n    0.0
weather_icon_50d    0.0
weather_icon_50n    0.0
Length: 76, dtype: float64

In [112]:
# drop missing values more than 50% 
weather_data = weather_data.loc[:, weather_data.isnull().mean() < 0.5]
weather_data


,date,time,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,...,weather_icon_09n,weather_icon_10,weather_icon_10d,weather_icon_10n,weather_icon_11d,weather_icon_11n,weather_icon_13d,weather_icon_13n,weather_icon_50d,weather_icon_50n
35145,2014-12-31,23,267.325,267.325,267.325,971,63,1,309,0.0,...,False,False,False,False,False,False,False,False,False,False
35146,2015-01-01,0,267.325,267.325,267.325,971,63,1,309,0.0,...,False,False,False,False,False,False,False,False,False,False
35147,2015-01-01,1,266.186,266.186,266.186,971,64,1,273,0.0,...,False,False,False,False,False,False,False,False,False,False
35148,2015-01-01,2,266.186,266.186,266.186,971,64,1,273,0.0,...,False,False,False,False,False,False,False,False,False,False
35149,2015-01-01,3,266.186,266.186,266.186,971,64,1,273,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71407,2018-12-31,18,283.560,282.150,285.150,1030,88,1,280,0.0,...,False,False,False,False,False,False,False,False,False,False
71408,2018-12-31,19,280.120,278.150,281.150,1031,52,1,260,0.0,...,False,False,False,False,False,False,False,False,False,False
71409,2018-12-31,20,278.150,278.150,278.150,1030,65,1,340,0.0,...,False,False,False,False,False,False,False,False,False,False
71410,2018-12-31,21,276.570,276.150,277.150,1031,69,2,340,0.0,...,False,False,False,False,False,False,False,False,False,False


#### Oil Price Data analysis

In [22]:
oil_data_raw = pd.read_excel('./Data/RBRTEd.xls', sheet_name='Data 1')

oil_data_raw['date'] = pd.to_datetime(oil_data_raw['Date'])
oil_data_raw['oil_price'] = oil_data_raw['Europe Brent Spot Price FOB (Dollars per Barrel)']




In [23]:
oil_data_raw

,Date,Europe Brent Spot Price FOB (Dollars per Barrel),date,oil_price
0,1987-05-20,18.63,1987-05-20,18.63
1,1987-05-21,18.45,1987-05-21,18.45
2,1987-05-22,18.55,1987-05-22,18.55
3,1987-05-25,18.60,1987-05-25,18.60
4,1987-05-26,18.63,1987-05-26,18.63
...,...,...,...,...
9458,2024-08-28,80.04,2024-08-28,80.04
9459,2024-08-29,81.55,2024-08-29,81.55
9460,2024-08-30,80.20,2024-08-30,80.20
9461,2024-09-02,77.82,2024-09-02,77.82


In [115]:
# imputing missing days in the oil data as the average of the previous and next day prices

oil_data_raw = oil_data_raw.sort_values(by='date')
oil_data_raw['imputed'] = False
all_days = pd.date_range(start=oil_data_raw['date'].min(), end=oil_data_raw['date'].max(), freq='D')
missing_days = all_days.difference(oil_data_raw['date'])


imputed_rows = []
for day in missing_days:
    previous_day_price = oil_data_raw.loc[(oil_data_raw['date'] < day) & (~oil_data_raw['date'].isin(missing_days)), 'oil_price'].iloc[-1]
    next_day_price = oil_data_raw.loc[(oil_data_raw['date'] > day) & (~oil_data_raw['date'].isin(missing_days)), 'oil_price'].iloc[0]
    imputed_value = (previous_day_price + next_day_price) / 2
    imputed_rows.append({'date': day, 'oil_price': imputed_value, 'imputed': True})

imputed_df = pd.DataFrame(imputed_rows)
oil_data_raw = pd.concat([oil_data_raw, imputed_df], ignore_index=True)

oil_data_raw = oil_data_raw.sort_values(by='date')

print(oil_data_raw)


            date  oil_price  imputed
0     1987-05-20     18.630    False
1     1987-05-21     18.450    False
2     1987-05-22     18.550    False
9463  1987-05-23     18.575     True
9464  1987-05-24     18.575     True
...          ...        ...      ...
9460  2024-08-30     80.200    False
13620 2024-08-31     79.010     True
13621 2024-09-01     79.010     True
9461  2024-09-02     77.820    False
9462  2024-09-03     76.460    False

[13622 rows x 3 columns]


In [116]:
oil_data_raw.to_csv('./Data/oil_data_processed.csv', index=False)

### Merge and daset creation

In [117]:
num_split = 3
splits =[]

if num_split == 1 :
    splits = [12]
elif num_split == 2:
    splits = [9, 14]
elif num_split == 3:
    splits = [9,14,21]
elif num_split == 4:
    splits = [6, 12, 18, 00]




In [118]:
weather_data.dtypes

date                datetime64[ns]
time                         int64
temp                       float64
temp_min                   float64
temp_max                   float64
                         ...      
weather_icon_11n              bool
weather_icon_13d              bool
weather_icon_13n              bool
weather_icon_50d              bool
weather_icon_50n              bool
Length: 76, dtype: object

In [119]:
energy_data.dtypes

date                                           datetime64[ns]
time                                                    int64
generation biomass                                    float64
generation fossil brown coal/lignite                  float64
generation fossil gas                                 float64
generation fossil hard coal                           float64
generation fossil oil                                 float64
generation hydro pumped storage consumption           float64
generation hydro run-of-river and poundage            float64
generation hydro water reservoir                      float64
generation nuclear                                    float64
generation other                                      float64
generation other renewable                            float64
generation solar                                      float64
generation waste                                      float64
generation wind onshore                               float64
forecast

In [120]:
# merge the energy and weather data on date and time
energy_weather_data = pd.merge(energy_data, weather_data, on=['date', 'time'], how='inner')
energy_weather_data

,date,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,...,weather_icon_09n,weather_icon_10,weather_icon_10d,weather_icon_10n,weather_icon_11d,weather_icon_11n,weather_icon_13d,weather_icon_13n,weather_icon_50d,weather_icon_50n
0,2014-12-31,23,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,...,False,False,False,False,False,False,False,False,False,False
1,2015-01-01,0,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,...,False,False,False,False,False,False,False,False,False,False
2,2015-01-01,1,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,...,False,False,False,False,False,False,False,False,False,False
3,2015-01-01,2,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,...,False,False,False,False,False,False,False,False,False,False
4,2015-01-01,3,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31,18,297.0,0.0,7634.0,2628.0,178.0,1.0,1135.0,4836.0,...,False,False,False,False,False,False,False,False,False,False
35060,2018-12-31,19,296.0,0.0,7241.0,2566.0,174.0,1.0,1172.0,3931.0,...,False,False,False,False,False,False,False,False,False,False
35061,2018-12-31,20,292.0,0.0,7025.0,2422.0,168.0,50.0,1148.0,2831.0,...,False,False,False,False,False,False,False,False,False,False
35062,2018-12-31,21,293.0,0.0,6562.0,2293.0,163.0,108.0,1128.0,2068.0,...,False,False,False,False,False,False,False,False,False,False


In [121]:
# merge the energy_weather_data and oil data on date
energy_weather_oil_data = pd.merge(energy_weather_data, oil_data_raw, on='date', how='inner')
energy_weather_oil_data

,date,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,...,weather_icon_10d,weather_icon_10n,weather_icon_11d,weather_icon_11n,weather_icon_13d,weather_icon_13n,weather_icon_50d,weather_icon_50n,oil_price,imputed
0,2014-12-31,23,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,...,False,False,False,False,False,False,False,False,55.270,False
1,2015-01-01,0,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,...,False,False,False,False,False,False,False,False,55.325,True
2,2015-01-01,1,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,...,False,False,False,False,False,False,False,False,55.325,True
3,2015-01-01,2,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,...,False,False,False,False,False,False,False,False,55.325,True
4,2015-01-01,3,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,...,False,False,False,False,False,False,False,False,55.325,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31,18,297.0,0.0,7634.0,2628.0,178.0,1.0,1135.0,4836.0,...,False,False,False,False,False,False,False,False,52.315,True
35060,2018-12-31,19,296.0,0.0,7241.0,2566.0,174.0,1.0,1172.0,3931.0,...,False,False,False,False,False,False,False,False,52.315,True
35061,2018-12-31,20,292.0,0.0,7025.0,2422.0,168.0,50.0,1148.0,2831.0,...,False,False,False,False,False,False,False,False,52.315,True
35062,2018-12-31,21,293.0,0.0,6562.0,2293.0,163.0,108.0,1128.0,2068.0,...,False,False,False,False,False,False,False,False,52.315,True


In [122]:
# select the row to be used in the model that are one with hours in the splits list

energy_weather_oil_data = energy_weather_oil_data[energy_weather_oil_data['time'].isin(splits)]
energy_weather_oil_data





,date,time,generation biomass,generation fossil brown coal/lignite,generation fossil gas,generation fossil hard coal,generation fossil oil,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,...,weather_icon_10d,weather_icon_10n,weather_icon_11d,weather_icon_11n,weather_icon_13d,weather_icon_13n,weather_icon_50d,weather_icon_50n,oil_price,imputed
10,2015-01-01,9,422.0,173.0,4059.0,3516.0,167.0,2020.0,1041.0,1817.0,...,False,False,False,False,False,False,False,False,55.325,True
15,2015-01-01,14,421.0,183.0,3708.0,4038.0,160.0,1069.0,1023.0,1151.0,...,False,False,False,False,False,False,False,False,55.325,True
22,2015-01-01,21,440.0,322.0,4870.0,4990.0,154.0,0.0,1178.0,5359.0,...,False,False,False,False,False,False,False,False,55.325,True
34,2015-01-02,9,377.0,0.0,2844.0,1339.0,200.0,938.0,1319.0,4289.0,...,False,False,False,False,False,False,False,False,55.380,False
39,2015-01-02,14,391.0,0.0,2599.0,1298.0,195.0,1491.0,1284.0,2984.0,...,False,False,False,False,False,False,False,False,55.380,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35031,2018-12-30,14,274.0,0.0,4826.0,1673.0,231.0,205.0,1036.0,1973.0,...,False,False,False,False,False,False,False,False,52.315,True
35038,2018-12-30,21,283.0,0.0,5660.0,1610.0,215.0,1.0,1160.0,3757.0,...,False,False,False,False,False,False,False,False,52.315,True
35050,2018-12-31,9,307.0,0.0,6484.0,2461.0,225.0,1.0,1054.0,2643.0,...,False,False,False,False,False,False,False,False,52.315,True
35055,2018-12-31,14,295.0,0.0,6128.0,2501.0,178.0,339.0,1052.0,1889.0,...,False,False,False,False,False,False,False,False,52.315,True


In [123]:
energy_weather_oil_data.columns

Index(['date', 'time', 'generation biomass',
       'generation fossil brown coal/lignite', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation nuclear',
       'generation other', 'generation other renewable', 'generation solar',
       'generation waste', 'generation wind onshore',
       'forecast solar day ahead', 'forecast wind onshore day ahead',
       'total load actual', 'price actual', 'temp', 'temp_min', 'temp_max',
       'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h',
       'snow_3h', 'clouds_all', 'weather_id', 'weather_main_clear',
       'weather_main_clouds', 'weather_main_drizzle', 'weather_main_fog',
       'weather_main_haze', 'weather_main_mist', 'weather_main_rain',
       'weather_main_snow', 'weather_main_thunderstorm',
       'weather_descrip

In [124]:
energy_weather_oil_data.columns

# drop the date column,    'forecast solar day ahead', 'forecast wind onshore day ahead','total load actual'
energy_weather_oil_data = energy_weather_oil_data.drop(['date', 'forecast solar day ahead', 'forecast wind onshore day ahead','total load actual'], axis=1)


In [125]:
# Create the column to predict that is the actual price of the next period (shift of -len(splits))
energy_weather_oil_data['target'] = energy_weather_oil_data['price actual'].shift(-len(splits))
energy_weather_oil_data[['time','price actual', 'target']]

,time,price actual,target
10,9,58.94,75.86
15,14,59.76,71.24
22,21,70.53,76.20
34,9,75.86,70.35
39,14,71.24,62.76
...,...,...,...
35031,14,66.97,70.85
35038,21,71.92,69.89
35050,9,72.12,NaN
35055,14,70.85,NaN


In [126]:
# drop the price actual column
energy_weather_oil_data = energy_weather_oil_data.drop(columns=['price actual'])


In [127]:
# X_train, X_test, y_train, y_test X_val, y_val 80% 10% 10%. y
from sklearn.model_selection import train_test_split

X = energy_weather_oil_data.drop(columns=['target'])
y = energy_weather_oil_data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# compare the shapes with tha data in /Data 

X_train.shape, X_test.shape, X_val.shape 




((3506, 91), (438, 91), (439, 91))

In [128]:
X_train_s = pd.read_csv('./Data/dataset/X_train.csv')
y_train_s = pd.read_csv('./Data/dataset/Y_train.csv').values.ravel()
X_val_s = pd.read_csv('./Data/dataset/X_val.csv')
y_val_s = pd.read_csv('./Data/dataset/Y_val.csv').values.ravel()
X_test_s = pd.read_csv('./Data/dataset/X_test.csv')
y_test_s = pd.read_csv('./Data/dataset/Y_test.csv').values.ravel()
random_state = 42

X_train_s.shape, X_test_s.shape, X_val_s.shape

((3496, 96), (437, 96), (437, 96))

# Experiments

### Baseline

In [25]:
X_train = pd.read_csv('./Data/dataset/X_train.csv')
y_train = pd.read_csv('./Data/dataset/Y_train.csv').values.ravel()
X_val = pd.read_csv('./Data/dataset/X_val.csv')
y_val = pd.read_csv('./Data/dataset/Y_val.csv').values.ravel()
X_test = pd.read_csv('./Data/dataset/X_test.csv')
y_test = pd.read_csv('./Data/dataset/Y_test.csv').values.ravel()
random_state = 42

In [130]:


y_train_mean = np.mean(y_train)

y_pred_train_baseline = np.full_like(y_train, y_train_mean)
y_pred_test_baseline = np.full_like(y_test, y_train_mean)
mae_train_baseline = mean_absolute_error(y_train, y_pred_train_baseline)
mae_test_baseline = mean_absolute_error(y_test, y_pred_test_baseline)
rmse_train_baseline = mean_squared_error(y_train, y_pred_train_baseline, squared=False)
rmse_test_baseline = mean_squared_error(y_test, y_pred_test_baseline, squared=False)

print('Baseline Model')
print('-------------------------------------------------------------------')
print('Training MAE:', mae_train_baseline)
print('-------------------------------------------------------------------')
print('Test MAE:', mae_test_baseline)
print('-------------------------------------------------------------------')
print('Training RMSE:', rmse_train_baseline)
print('-------------------------------------------------------------------')
print('Test RMSE:', rmse_test_baseline)
print('-------------------------------------------------------------------')

Baseline Model
-------------------------------------------------------------------
Training MAE: 10.780915362899213
-------------------------------------------------------------------
Test MAE: 11.268394313998606
-------------------------------------------------------------------
Training RMSE: 13.828830392810245
-------------------------------------------------------------------
Test RMSE: 14.332784856304427
-------------------------------------------------------------------


### LinearRegressor

In [131]:

linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
y_pred_train_linear = linear_regressor.predict(X_train)
y_pred_test_linear = linear_regressor.predict(X_test)
mae_train_linear = mean_absolute_error(y_train, y_pred_train_linear)
mae_test_linear = mean_absolute_error(y_test, y_pred_test_linear)
rmse_train_linear = mean_squared_error(y_train, y_pred_train_linear, squared=False)
rmse_test_linear = mean_squared_error(y_test, y_pred_test_linear, squared=False)

# Print the linear regression model results
print('Linear Regression Model')
print('-------------------------------------------------------------------')
print('Training MAE:', mae_train_linear)
print('-------------------------------------------------------------------')
print('Test MAE:', mae_test_linear)
print('-------------------------------------------------------------------')
print('Training RMSE:', rmse_train_linear)
print('-------------------------------------------------------------------')
print('Test RMSE:', rmse_test_linear)
print('-------------------------------------------------------------------')

Linear Regression Model
-------------------------------------------------------------------
Training MAE: 7.7485420814263986
-------------------------------------------------------------------
Test MAE: 7.337551886851522
-------------------------------------------------------------------
Training RMSE: 10.264544599636867
-------------------------------------------------------------------
Test RMSE: 9.878422954741376
-------------------------------------------------------------------


### Ridge Regression

In [132]:

def objective(trial):
    # Define the hyperparameters to be tuned
    alpha = trial.suggest_float('alpha', 1e-4, 1e2, log=True)
    
    # Create the Ridge regression model with the suggested hyperparameters
    model = Ridge(alpha=alpha, random_state=random_state)
    
    # Fit the model to the training data
    model.fit(X_train, y_train)
    
    # Predict on the validation set
    y_pred = model.predict(X_val)
    
    # Calculate the mean squared error on the validation set
    mse = mean_squared_error(y_val, y_pred)
    
    return mse

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
print("Best parameters:", best_params)

# Train the final model with the best hyperparameters
final_model = Ridge(**best_params, random_state=random_state)

X_train_val = pd.concat([X_train, X_val], axis=0)
y_train_val = np.concatenate([y_train, y_val])
final_model.fit(X_train_val, y_train_val)

# Evaluate the final model
print('Ridge Regression Model')
print('-------------------------------------------------------------------')
y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
print('Training MAE:', mae_train)
print('-------------------------------------------------------------------')
print('Test MAE:', mae_test)
print('-------------------------------------------------------------------')
print('Training RMSE:', rmse_train)
print('-------------------------------------------------------------------')
print('Test RMSE:', rmse_test)
print('-------------------------------------------------------------------')

[I 2024-09-12 14:02:44,976] A new study created in memory with name: no-name-dbe29255-4dba-422b-978c-df9700f34582
[I 2024-09-12 14:02:45,027] Trial 0 finished with value: 111.48444625634049 and parameters: {'alpha': 41.51105209365247}. Best is trial 0 with value: 111.48444625634049.
[I 2024-09-12 14:02:45,099] Trial 1 finished with value: 111.24913602467349 and parameters: {'alpha': 0.1695810822363773}. Best is trial 1 with value: 111.24913602467349.
[I 2024-09-12 14:02:45,124] Trial 2 finished with value: 111.02487532779998 and parameters: {'alpha': 1.4807980230475328}. Best is trial 2 with value: 111.02487532779998.
[I 2024-09-12 14:02:45,137] Trial 3 finished with value: 111.31063038446935 and parameters: {'alpha': 0.07984984971609645}. Best is trial 2 with value: 111.02487532779998.
[I 2024-09-12 14:02:45,151] Trial 4 finished with value: 111.00277567545677 and parameters: {'alpha': 2.677893130014894}. Best is trial 4 with value: 111.00277567545677.
[I 2024-09-12 14:02:45,164] Tria

Best parameters: {'alpha': 3.019726390542566}
Ridge Regression Model
-------------------------------------------------------------------
Training MAE: 7.7454044311336485
-------------------------------------------------------------------
Test MAE: 7.323957152243949
-------------------------------------------------------------------
Training RMSE: 10.273098097744814
-------------------------------------------------------------------
Test RMSE: 9.877096616526849
-------------------------------------------------------------------


### Random Forest

In [28]:
random_state = 42

def objective(trial):

    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=random_state
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    return mse

#study = optuna.create_study(direction='minimize')
#study.optimize(objective, n_trials=400)


#best_params = study.best_params
#print("Best parameters:", best_params)


#final_model = RandomForestRegressor(**best_params, random_state=random_state)

# save the model
#joblib.dump(final_model, '.Data/RandomForestmodel.pkl.pkl')
# Load the model
final_model = joblib.load('./Data/RandomForestmodel.pkl')



X_train_val = pd.concat([X_train, X_val], axis=0)
y_train_val = np.concatenate([y_train, y_val])
final_model.fit(X_train_val, y_train_val)



# Evaluate the final model
print('Random Forest Model')
print('-------------------------------------------------------------------')
y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
print('Training MAE:', mae_train)
print('-------------------------------------------------------------------')
print('Test MAE:', mae_test)
print('-------------------------------------------------------------------')
print('Training RMSE:', rmse_train)
print('-------------------------------------------------------------------')
print('Test RMSE:', rmse_test)
print('-------------------------------------------------------------------')

Random Forest Model
-------------------------------------------------------------------
Training MAE: 2.3497902518511435
-------------------------------------------------------------------
Test MAE: 5.900074981698189
-------------------------------------------------------------------
Training RMSE: 3.163844253948153
-------------------------------------------------------------------
Test RMSE: 7.951572934575552
-------------------------------------------------------------------


### XGBOOST

In [78]:
def objective(trial):
    # Define the hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True)
    }
    
   
    model = XGBRegressor(**params, random_state=random_state)
    

    model.fit(X_train, y_train)
    

    y_pred = model.predict(X_val)
    
    
    mse = mean_squared_error(y_val, y_pred)
    
    return mse

# Uncomment these lines to perform hyperparameter tuning
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# Get the best hyperparameters
# best_params = study.best_params
# print("Best parameters:", best_params)

# Train the final model with the best hyperparameters
# final_model = XGBRegressor(**best_params, random_state=random_state)

# Save the model
# joblib.dump(final_model, './Data/XGBRmodel.pkl')

# Load the model
final_model = joblib.load('./Data/XGBRmodel.pkl')

X_train_val = pd.concat([X_train, X_val], axis=0)
y_train_val = np.concatenate([y_train, y_val])
final_model.fit(X_train_val, y_train_val)

# Evaluate the final model
print('XGBoost Model')
print('-------------------------------------------------------------------')
y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
print('Training MAE:', mae_train)
print('-------------------------------------------------------------------')
print('Test MAE:', mae_test)
print('-------------------------------------------------------------------')
print('Training RMSE:', rmse_train)
print('-------------------------------------------------------------------')
print('Test RMSE:', rmse_test)
print('-------------------------------------------------------------------')

XGBoost Model
-------------------------------------------------------------------
Training MAE: 1.1718655350442881
-------------------------------------------------------------------
Test MAE: 5.466208830774513
-------------------------------------------------------------------
Training RMSE: 1.573099821833715
-------------------------------------------------------------------
Test RMSE: 7.36635478022455
-------------------------------------------------------------------


### GP

In [39]:
from gplearn.genetic import SymbolicRegressor
with open('Hyperparameters/gp.json', 'r') as f:
    best_params = json.load(f)


random_state = 42



p_crossover = best_params['p_crossover']
p_subtree_mutation = best_params['p_subtree_mutation']
p_hoist_mutation = best_params['p_hoist_mutation']
p_point_mutation = best_params['p_point_mutation']


prob_sum = p_crossover + p_subtree_mutation + p_hoist_mutation + p_point_mutation

p_crossover_norm = (p_crossover / prob_sum) -0.000001
p_subtree_mutation_norm = (p_subtree_mutation / prob_sum) -0.000001
p_hoist_mutation_norm = (p_hoist_mutation / prob_sum) -0.000001
p_point_mutation_norm = (p_point_mutation / prob_sum) -0.000001




best_params['p_crossover'] = p_crossover_norm
best_params['p_subtree_mutation'] = p_subtree_mutation_norm
best_params['p_hoist_mutation'] = p_hoist_mutation_norm
best_params['p_point_mutation'] = p_point_mutation_norm


final_model = SymbolicRegressor(**best_params, random_state=random_state)

X_train_val = pd.concat([X_train, X_val], axis=0)
y_train_val = np.concatenate([y_train, y_val], axis=0)

final_model.fit(X_train_val, y_train_val)

joblib.dump(final_model, 'GP.pkl')

y_pred_train = final_model.predict(X_train)
y_pred_test = final_model.predict(X_test)

mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
print('Training MAE:', mae_train)
print('-------------------------------------------------------------------')
print('Test MAE:', mae_test)
print('-------------------------------------------------------------------')
print('Training RMSE:', rmse_train)
print('-------------------------------------------------------------------')
print('Test RMSE:', rmse_test)
print('-------------------------------------------------------------------')


Training MAE: 8.41942726989568
-------------------------------------------------------------------
Test MAE: 8.134973328640527
-------------------------------------------------------------------
Training RMSE: 11.30135896707606
-------------------------------------------------------------------
Test RMSE: 10.887546402874916
-------------------------------------------------------------------


### GSGP

In [45]:

X_train = pd.read_csv('./Data/dataset/X_train.csv')
num_variable = X_train.shape[1]
X_train = pd.read_csv('./Data/dataset/X_train.csv').values
y_train = pd.read_csv('./Data/dataset/Y_train.csv').values.ravel()
X_val = pd.read_csv('./Data/dataset/X_val.csv').values
y_val = pd.read_csv('./Data/dataset/Y_val.csv').values.ravel()
X_test = pd.read_csv('./Data/dataset/X_test.csv').values
y_test = pd.read_csv('./Data/dataset/Y_test.csv').values.ravel()
import numpy as np
import random
import math
from typing import List, Callable

class Symbol:
    def __init__(self, is_function: bool, arity: int, name: str, value: float = 0.0):
        self.is_function = is_function
        self.arity = arity
        self.name = name
        self.value = value

class Node:
    def __init__(self, root: Symbol, children: List['Node'] = None):
        self.root = root
        self.children = children or []

class Individual:
    def __init__(self, root: Node):
        self.root = root
        self.fitness = float('inf')

class SymbolicRegressor:
    def __init__(self, params: dict):
        self.params = params
        self.symbols = self.create_symbols()
        self.population = []
        self.best_individual = None

    def create_symbols(self):
        symbols = [
            Symbol(True, 2, "+"),
            Symbol(True, 2, "-"),
            Symbol(True, 2, "*"),
            Symbol(True, 2, "/")
        ]
        symbols.extend([Symbol(False, 0, f"x{i}") for i in range(self.params['num_variables'])])
        symbols.extend([Symbol(False, 0, str(random.uniform(self.params['min_constant'], self.params['max_constant']))) 
                        for _ in range(self.params['num_constants'])])
        return symbols

    def create_tree(self, depth: int, max_depth: int) -> Node:
        if depth == max_depth or (depth > 0 and random.random() < 0.5):
            return Node(random.choice([s for s in self.symbols if not s.is_function]))
        
        root = random.choice([s for s in self.symbols if s.is_function])
        children = [self.create_tree(depth + 1, max_depth) for _ in range(root.arity)]
        return Node(root, children)

    def create_population(self):
        self.population = [Individual(self.create_tree(0, self.params['max_depth'])) 
                           for _ in range(self.params['population_size'])]

    def evaluate(self, individual: Individual, X: np.ndarray, y: np.ndarray) -> float:
        predictions = np.array([self.eval_tree(individual.root, x) for x in X])
        return np.sqrt(np.mean((predictions - y) ** 2))

    def eval_tree(self, node: Node, x: np.ndarray) -> float:
        if not node.root.is_function:
            return float(node.root.name) if node.root.name[0] != 'x' else x[int(node.root.name[1:])]
        
        if len(node.children) == 0:
            return 0  
        
        if len(node.children) == 1:
            return self.eval_tree(node.children[0], x)  
        
        left = self.eval_tree(node.children[0], x)
        right = self.eval_tree(node.children[1], x) if len(node.children) > 1 else 0

        if node.root.name == '+': return left + right
        if node.root.name == '-': return left - right
        if node.root.name == '*': return left * right
        if node.root.name == '/': return left / right if right != 0 else 1

    def tournament_selection(self) -> Individual:
        tournament = random.sample(self.population, self.params['tournament_size'])
        return min(tournament, key=lambda ind: ind.fitness)

    def crossover(self, parent1: Individual, parent2: Individual) -> Individual:
        def swap_subtree(node1: Node, node2: Node) -> Node:
            if random.random() < 0.5:
                return Node(node2.root, node2.children)
            if not node1.root.is_function:
                return Node(node1.root)
            new_children = [swap_subtree(c1, c2) for c1, c2 in zip(node1.children, node2.children)]
            return Node(node1.root, new_children)
        
        new_root = swap_subtree(parent1.root, parent2.root)
        return Individual(new_root)

    def mutation(self, individual: Individual) -> Individual:
        def mutate_node(node: Node) -> Node:
            if random.random() < self.params['mutation_rate']:
                return self.create_tree(0, self.params['max_depth'])
            if not node.root.is_function:
                return Node(node.root)
            new_children = [mutate_node(child) for child in node.children]
            return Node(node.root, new_children)
        
        new_root = mutate_node(individual.root)
        return Individual(new_root)

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.create_population()
        for generation in range(self.params['num_generations']):
            for ind in self.population:
                ind.fitness = self.evaluate(ind, X, y)
            
            self.best_individual = min(self.population, key=lambda ind: ind.fitness)
            new_population = [self.best_individual]  # Elitism

            while len(new_population) < self.params['population_size']:
                if random.random() < self.params['crossover_rate']:
                    parent1, parent2 = self.tournament_selection(), self.tournament_selection()
                    offspring = self.crossover(parent1, parent2)
                else:
                    offspring = self.mutation(self.tournament_selection())
                new_population.append(offspring)

            self.population = new_population

            if generation % 10 == 0:  # Print progress every 10 generations
                print(f"Generation {generation}: Best Fitness = {self.best_individual.fitness}")

        return self.best_individual.fitness

    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.array([self.eval_tree(self.best_individual.root, x) for x in X])

with open('Hyperparameters/gpgs.json', 'r') as f:
    best_params = json.load(f)

best_params['num_constant'] = num_variable

best_regressor = SymbolicRegressor(best_params)
best_regressor.fit(X_train, y_train)


y_pred_test = best_regressor.predict(X_test)
y_pred_train = best_regressor.predict(X_train)

mae_train = np.mean(np.abs(y_train - y_pred_train))
rmse_train = np.sqrt(np.mean((y_train - y_pred_train) ** 2))
mae_test = np.mean(np.abs(y_test - y_pred_test))
rmse_test = np.sqrt(np.mean((y_test - y_pred_test) ** 2))


print('GSGP Model')
print('-------------------------------------------------------------------')
print('Training MAE:', mae_train)
print('-------------------------------------------------------------------')
print('Test MAE:', mae_test)
print('-------------------------------------------------------------------')
print('Training RMSE:', rmse_train)
print('-------------------------------------------------------------------')
print('Test RMSE:', rmse_test)
print('-------------------------------------------------------------------')




Generation 0: Best Fitness = 29.517975625710612
Generation 10: Best Fitness = 15.55880441678803
Generation 20: Best Fitness = 14.167745469150073
Generation 30: Best Fitness = 14.028618807373501
Generation 40: Best Fitness = 13.312201145056635
Generation 50: Best Fitness = 13.30157987914756
Generation 60: Best Fitness = 13.285071706374017
Generation 70: Best Fitness = 13.285071706374017
Generation 80: Best Fitness = 13.278134423141138
Generation 90: Best Fitness = 13.278134423141138
Generation 100: Best Fitness = 13.278028115368357
Generation 110: Best Fitness = 13.091382575373466


### LSGP

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.preprocessing import StandardScaler
import optuna
import pandas as pd
from typing import List, Callable, Union
from abc import ABC, abstractmethod

# Load and preprocess data
def load_and_preprocess_data(train_path: str, val_path: str, test_path: str):
    X_train = pd.read_csv(f'{train_path}/X_train.csv')
    y_train = pd.read_csv(f'{train_path}/Y_train.csv').iloc[:, 0]
    X_val = pd.read_csv(f'{val_path}/X_val.csv')
    y_val = pd.read_csv(f'{val_path}/Y_val.csv').iloc[:, 0]
    X_test = pd.read_csv(f'{test_path}/X_test.csv')
    y_test = pd.read_csv(f'{test_path}/Y_test.csv').iloc[:, 0]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, y_train, X_val, y_val, X_test, y_test

# Load data
X_train, y_train, X_val, y_val, X_test, y_test = load_and_preprocess_data(
    './Data/dataset', './Data/dataset', './Data/dataset'
)

class Node(ABC):
    @abstractmethod
    def evaluate(self, X: np.ndarray) -> np.ndarray:
        pass

    @abstractmethod
    def update(self, error: np.ndarray, X: np.ndarray, learning_rate: float) -> 'Node':
        pass

class Tree(Node):
    def __init__(self, op: Callable, left: Node, right: Node):
        self.op = op
        self.left = left
        self.right = right

    def evaluate(self, X: np.ndarray) -> np.ndarray:
        return self.op(self.left.evaluate(X), self.right.evaluate(X))

    def update(self, error: np.ndarray, X: np.ndarray, learning_rate: float) -> 'Tree':
        return Tree(self.op, 
                    self.left.update(error, X, learning_rate),
                    self.right.update(error, X, learning_rate))

class Variable(Node):
    def __init__(self, idx: int):
        self.idx = idx

    def evaluate(self, X: np.ndarray) -> np.ndarray:
        return X[:, self.idx]

    def update(self, error: np.ndarray, X: np.ndarray, learning_rate: float) -> 'Variable':
        gradient = np.mean(error * X[:, self.idx])
        new_idx = max(0, min(X.shape[1] - 1, self.idx - int(np.sign(gradient))))
        return Variable(new_idx)

class Constant(Node):
    def __init__(self, value: float):
        self.value = value

    def evaluate(self, X: np.ndarray) -> np.ndarray:
        return np.full(X.shape[0], self.value)

    def update(self, error: np.ndarray, X: np.ndarray, learning_rate: float) -> 'Constant':
        gradient = np.mean(error)
        new_value = self.value - learning_rate * gradient
        return Constant(new_value)

def safe_div(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.divide(a, b, out=np.ones_like(a), where=np.abs(b)>=1e-8)

class SymbolicRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, population_size: int = 100, tournament_size: int = 4, crossover_prob: float = 0.9,
                 mutation_prob: float = 0.3, mutation_step: float = 0.1, num_generations: int = 250,
                 num_gen_1: int = 125, num_gen_2: int = 125, learning_rate: float = 0.0001, algorithm: str = 'HeH-GSGP'):
        self.population_size = population_size
        self.tournament_size = tournament_size
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.mutation_step = mutation_step
        self.num_generations = num_generations
        self.num_gen_1 = num_gen_1
        self.num_gen_2 = num_gen_2
        self.learning_rate = learning_rate
        self.algorithm = algorithm
        self.ops: List[Callable] = [np.add, np.subtract, np.multiply, safe_div]
        self.consts: List[float] = [-1.0, 1.0]

    def _random_tree(self, n_vars: int, max_depth: int, end_prob: float) -> Node:
        if max_depth == 0 or np.random.random() < end_prob:
            if np.random.random() > 0.5:
                return Variable(np.random.randint(n_vars))
            else:
                return Constant(np.random.choice(self.consts))
        else:
            left = self._random_tree(n_vars, max_depth - 1, end_prob)
            right = self._random_tree(n_vars, max_depth - 1, end_prob)
            return Tree(np.random.choice(self.ops), left, right)

    def _ramped_half_half(self, n_vars: int, max_depth: int) -> List[Node]:
        return [self._random_tree(n_vars, np.random.randint(1, max_depth + 1), 0.5) 
                for _ in range(self.population_size)]

    def _rmse(self, y: np.ndarray, y_pred: np.ndarray) -> float:
        return np.sqrt(np.mean((y - y_pred) ** 2))

    def _tournament(self, fit: List[float]) -> int:
        tournament_size = min(self.tournament_size, len(fit))
        idx = np.random.choice(len(fit), tournament_size, replace=False)
        return idx[np.argmin([fit[i] for i in idx])]

    def _next_generation(self, population: List[Node], X: np.ndarray, y: np.ndarray) -> List[Node]:
        fit = [self._rmse(y, ind.evaluate(X)) for ind in population]
        new_population = []
        for _ in range(self.population_size):
            if np.random.random() < self.crossover_prob:
                parent1 = population[self._tournament(fit)]
                parent2 = population[self._tournament(fit)]
                alpha = np.random.random()
                child = Tree(np.add, 
                             Tree(np.multiply, Constant(alpha), parent1),
                             Tree(np.multiply, Constant(1-alpha), parent2))
            else:
                child = population[self._tournament(fit)]
            
            if np.random.random() < self.mutation_prob:
                random_tree = self._random_tree(self.n_features_in_, 2, 0.5)
                child = Tree(np.add, child, 
                             Tree(np.multiply, Constant(self.mutation_step), random_tree))
            
            new_population.append(child)
        
        best_idx = np.argmin(fit)
        new_population[0] = population[best_idx]
        
        return new_population

    def _gradient_descent(self, population: List[Node], X: np.ndarray, y: np.ndarray) -> List[Node]:
        new_population = []
        for ind in population:
            y_pred = ind.evaluate(X)
            error = y_pred - y
            new_ind = ind.update(error, X, self.learning_rate)
            new_population.append(new_ind)
        
        old_fit = [self._rmse(y, ind.evaluate(X)) for ind in population]
        new_fit = [self._rmse(y, ind.evaluate(X)) for ind in new_population]
        
        final_population = [new_population[i] if new_fit[i] < old_fit[i] else population[i] 
                            for i in range(len(population))]
        
        return final_population

    def _calculate_population_loss(self, population: List[Node], X: np.ndarray, y: np.ndarray) -> float:
        losses = [self._rmse(y, ind.evaluate(X)) for ind in population]
        return np.mean(losses)

    def _heh_gsgp(self, population: List[Node], X: np.ndarray, y: np.ndarray) -> List[Node]:
        for i in range(self.num_gen_1):
            population = self._next_generation(population, X, y)
            loss = self._calculate_population_loss(population, X, y)
            print(f"HeH-GSGP Genetic step {i+1}/{self.num_gen_1}, Loss: {loss}")
        
        for i in range(self.num_gen_2):
            population = self._gradient_descent(population, X, y)
            loss = self._calculate_population_loss(population, X, y)
            print(f"HeH-GSGP Gradient step {i+1}/{self.num_gen_2}, Loss: {loss}")
        
        return population

    def _hyb_gsgp(self, population: List[Node], X: np.ndarray, y: np.ndarray) -> List[Node]:
        for i in range(self.num_generations):
            population = self._next_generation(population, X, y)
            population = self._gradient_descent(population, X, y)
            loss = self._calculate_population_loss(population, X, y)
            print(f"HYB-GSGP Generation {i+1}/{self.num_generations}, Loss: {loss}")
        
        return population

    def fit(self, X: np.ndarray, y: np.ndarray) -> 'SymbolicRegressor':
        X, y = check_X_y(X, y)
        self.n_features_in_ = X.shape[1]

        population = self._ramped_half_half(self.n_features_in_, 4)
        initial_loss = self._calculate_population_loss(population, X, y)
        print(f"Initial population loss: {initial_loss}")

        if self.algorithm == 'HeH-GSGP':
            population = self._heh_gsgp(population, X, y)
        elif self.algorithm == 'HYB-GSGP':
            population = self._hyb_gsgp(population, X, y)
        else:
            raise ValueError("Algorithm must be either 'HeH-GSGP' or 'HYB-GSGP'")

        fit = [self._rmse(y, ind.evaluate(X)) for ind in population]
        self.best_individual_ = population[np.argmin(fit)]
        final_loss = self._calculate_population_loss([self.best_individual_], X, y)
        print(f"Final best individual loss: {final_loss}")

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        check_is_fitted(self)
        X = check_array(X)
        return self.best_individual_.evaluate(X)

def objective(trial):
    algorithm = ['HeH-GSGP', 'HYB-GSGP'][trial.number % 2]
    params = {
        'population_size': trial.suggest_int('population_size', 20, 200),
        'tournament_size': trial.suggest_int('tournament_size', 2, 10),
        'crossover_prob': trial.suggest_float('crossover_prob', 0.1, 1.0),
        'mutation_prob': trial.suggest_float('mutation_prob', 0.1, 0.5),
        'mutation_step': trial.suggest_float('mutation_step', 0.01, 0.5),
        'num_generations': trial.suggest_int('num_generations', 10, 50),
        'num_gen_1': trial.suggest_int('num_gen_1', 5, 25),
        'num_gen_2': trial.suggest_int('num_gen_2', 5, 25),
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
        'algorithm': algorithm
    }
    
    model = SymbolicRegressor(**params)
    model.fit(X_train, y_train)
    
    y_pred_val = model.predict(X_val)
    mse_val = np.mean((y_val - y_pred_val) ** 2)
    
    return mse_val

if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)  

    print("Best parameters:", study.best_params)

    
    best_model = SymbolicRegressor(**study.best_params)
    best_model.fit(X_train, y_train)

    # Evaluate on test set
    y_pred_test = best_model.predict(X_test)
    mse_test = np.mean((y_test - y_pred_test) ** 2)
    print(f"Mean Squared Error on Test Set: {mse_test}")